In [21]:
from bokeh.io import output_notebook 
from bokeh.plotting import figure, output_file, show 
from bokeh.palettes import brewer, colorblind
import ee

In [4]:
ee.Authenticate()

Enter verification code:  4/1AY0e-g4RRzYeEFJuczFKyvwjqpNWH-gkoJpYieup1ZcX-81qE-Q4yqDJDJY



Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
# SMAP soil moisture "profile"/fraction (0-1 unitless) of estimated capacity
smp = (
    ee
    .ImageCollection('NASA_USDA/HSL/SMAP_soil_moisture')
    .select('smp')
    .first())

In [36]:
# palette_name = 'Colorblind'
# palette_len = 4
# palette = colorblind[palette_name][palette_len]

palette_name = 'BrBG'
palette_len = 11
palette = brewer[palette_name][palette_len][::-1]
vis_params = {
    'min': 0, 'max': 1, 'dimensions': 512,
    'palette': palette}

In [43]:
output_notebook()
graph = figure(title=palette_name)  
# demonstrating the BrBG palette 
graph.vbar(x = range(palette_len),  
           top = [1] * palette_len, 
           bottom = [0] * palette_len, 
           width = 1, 
           color = palette) 
show(graph)

Loading BokehJS ...

In [38]:
# Display a thumbnail
from IPython.display import Image
Image(url = smp.getThumbURL(vis_params))

In [39]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [40]:
# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(smp, vis_params, 'Soil Moisture Fraction')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)